# SmolTTS

In [ ]:
from smoltts_mlx import SmolTTS

model = SmolTTS()

In [ ]:
from IPython.display import Audio

pcm = model("At the moment, we do not support quantization for the PyTorch version, so you will need a GPU with a significant amount of memory.", voice="heart")
Audio(pcm, rate=model.sampling_rate)